# GANによる画像生成のメカニズムとDCGANの実装

commit用

```mycode/5_gan_generation/```

In [1]:
!pip list | grep scikit

scikit-image                      0.16.2             
scikit-learn                      0.22.2.post1       


In [2]:
!gpustat

s2                   Tue Apr 14 12:08:16 2020  440.44
[0] Quadro RTX 8000  | 65'C, 100 % | 11338 / 48601 MB | kera(733M) taoli(10589M)
[1] Quadro RTX 8000  | 26'C,   0 % |    12 / 48601 MB |
[2] Quadro RTX 8000  | 68'C, 100 % | 17045 / 48601 MB | taoli(17031M)
[3] Quadro RTX 8000  | 63'C, 100 % | 22401 / 48601 MB | taoli(22385M)
[4] Quadro RTX 8000  | 62'C,  98 % |  8859 / 48601 MB | hara(8847M)
[5] Quadro RTX 8000  | 64'C,  80 % | 29181 / 48601 MB | taoli(29165M)
[6] Quadro RTX 8000  | 62'C,  98 % |  6263 / 48601 MB | taoli(6249M)
[7] Quadro RTX 8000  | 65'C,  90 % | 16319 / 48601 MB | taoli(16305M)


In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [4]:
# パッケージのimport
import random
import math
import time
import pandas as pd
import numpy as np
from PIL import Image

import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import transforms
%config IPCompleter.greedy=True

## 5-1 GANによる画像生成のメカニズムとDCGANの実装

### Generatorのメカニズム
Generatorは入力乱数から目的の画像を生成するのでデータの次元を拡大していく必要がある．
このときに重要になるのが**転置畳み込み**である．
普通の畳み込みは入力データの全てのセルとカーネルの全てのセルの計算結果を集約して１つのセルにするのに対して，転置畳み込みは１つの入力セルに対して全てのカーネルのセルを作用させてその１つ１つを出力とするので結果として出力されるデータの次元は大きくなる．

### Generatorの実装
本節ではGANの中でも特に**DCGAN**と呼ばれるものを実装する．

***DCGAN：***Deep Convolution Generative Adversarial Network

実装に当たって入力する乱数の次元を```z_dim=20```にしてるがこれには特に意味はなく，生成画像の多様性が担保されればなんでも良い


In [7]:
class Generator(nn.Module):
    
    def __init__(self, z_dim = 20, image_size=64):
        super(Generator, self).__init__()
        
        # ここで out_channels = image_size*8に特別な意味は？
        self.layer1 = nn.Sequential(
            nn.ConvTranspose2d(z_dim, image_size*8, kernel_size=4, stride=1),
            nn.BatchNorm2d(image_size*8),
            nn.ReLu(inplace=True))
        
        self.layer2 = nn.Sequential(
            nn.ConvTranspose2d(image_size*8, image_size*4, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(image_size*4),
            nn.ReLu(inplace=True))
            
        self.layer2 = nn.Sequential(
            nn.ConvTranspose2d(image_size*8, image_size*4, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(image_size*4),
            nn.ReLu(inplace=True))
        
        self.layer3 = nn.Sequential(
            nn.ConvTranspose2d(image_size*4, image_size*2, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(image_size*2),
            nn.ReLu(inplace=True))
              
        self.layer4 = nn.Sequential(
            nn.ConvTranspose2d(image_size*2, image_size, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(image_size),
            nn.ReLu(inplace=True))
        
        self.last = nn.Sequential(
            nn.ConvTranspose2d(image_size, 1, kernel_size=4, stride=2, padding=1),
            nn.Tanh())
        # 白黒なので出力は1チャネル
        
        
    def forward(self, z):
        out = self.layer1(z)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.last(out)
        
        return out

In [8]:
# 実装したネットワークで出力
import matplotlib.pyplot as plt
%matplotlib inline

G = Generator(z_dim=20, image_size=64)